In [1]:
import random as rd
import numpy as np
import math

In [2]:
def init_population(_mu:int = 20, n:int = 8):
    "Inicia uma população com o tamanho das peças do tabuleiro e a quantidade de rainhas como parâmetro"
    population = []
    for i in range (_mu):
        population.append(rd.sample(range(n), n))

    return population


In [3]:
def fitness_nq(solution):
    """Informa a quantidade de cheques em cada solução"""
    xeques = 0
    for i in range(0,len(solution)):
        for j in range(0,len(solution)):
            if i!=j:
                if i-solution[i] == j-solution[j] or i+solution[i] == j+solution[j]:
                    xeques+=1

    return xeques

In [4]:
def selection(pop: list, n_rainhas: int):
    "Retorna as duas melhores soluções (melhores fitness)"
    
    pop_fitness = [fitness_nq(each_solution) for each_solution in pop]

    min_xeques_1 = max(pop_fitness)
    min_xeques_2 = min(pop_fitness)
    position_2 = n_rainhas + 1
    position_1 = n_rainhas + 1

    for i, num_xeques in enumerate(pop_fitness):
        if num_xeques ==  min_xeques_2 and position_2 != i:
            position_2 = i
        elif num_xeques < min_xeques_1:
            min_xeques_1 = num_xeques
            position_1 = i
            
    melhores_2 = [pop[position_1], pop[position_2]]
    return melhores_2

In [5]:
def crossover(subset_parents, crossover_rate):
    "Retorna um subset (dois indivíduos filhos) como resultado do cruzamento dos pais"
    len_parents = len(subset_parents[0])
    cut = int(1+len_parents*crossover_rate) # Valor onde irá cortar
    
    # Fazendo a combinação entre os pais para gerar os filhos
    son_1 = subset_parents[0].copy()
    son_2 = subset_parents[1].copy()
    
    son_1_genes = son_1[cut:]
    son_1[cut:] = son_2[cut:]
    son_2[cut:] = son_1_genes
    
    return [son_1, son_2]

In [6]:
def mutation(offspring, mutation_rate, n_rainhas):
    "Retorna um subset (dois indivíduos) como resultado da mutação dos filhos"
    size_ind = len(offspring[0])
    quant_mut = int(size_ind*mutation_rate)
    index_mult = np.linspace(0, size_ind-1, quant_mut)
    
    new_springs = []
    for spring in offspring:
        for i in index_mult:
            i = int(i)
            new_number = rd.randint(0, n_rainhas-1)
            spring[i] = new_number
            
        new_springs.append(spring)

    return new_springs

In [7]:
def replacement(offspring_new, pop):
    "Atualiza a lista de população, substituindo os dois piores individuos pelos dois que fazem parte do subset offspring new"
    
    pop_fitness = [fitness_nq(each_solution) for each_solution in pop]

    max_xeques_1 = max(pop_fitness)
    max_xeques_2 = min(pop_fitness)
    position_1 = n_rainhas + 1
    position_2 = n_rainhas + 1

    for i, num_xeques in enumerate(pop_fitness):
        if num_xeques ==  max_xeques_1 and position_1 != i:
            position_1 = i
        elif num_xeques > max_xeques_2:
            max_xeques_2 = num_xeques
            position_2 = i
    # print("----------------------------------------------------------------")
    pop_fitness = [fitness_nq(each_solution) for each_solution in pop]
    # print(pop_fitness)
    pop[position_1] = offspring_new[0]
    pop[position_2] = offspring_new[1]
    pop_fitness = [fitness_nq(each_solution) for each_solution in pop]
    # print(pop_fitness)
    return pop

In [8]:
def evalution(pop: list):
    "Retorna o melhor resultado e a média de xeques"
    pop_fitness = [fitness_nq(each_solution) for each_solution in pop]
    fitness_best = min(pop_fitness)
    fitness_mean = np.mean(pop_fitness)
    return fitness_best, fitness_mean

In [14]:
i_geracoes = 0

crossover_rate = 0.5
mutation_rate = 0.8 
n_rainhas = 8
tamanho_tabuleiro = 20
num_epochs = 10
fitness_best = None

list_best_fitness = []
list_mean_fitness = []

pop = init_population(tamanho_tabuleiro, n_rainhas)

print("Melhor Resultado | Resultado Médio")
while i_geracoes <= num_epochs and fitness_best != 0:
    subset_parents = selection(pop, n_rainhas)
    offspring = crossover(subset_parents, crossover_rate)
    pop_fitness = [fitness_nq(each_solution) for each_solution in pop]
    offspring_new = mutation(offspring, mutation_rate, n_rainhas)
    
    pop = replacement(offspring_new, pop)
    pop_fitness = [fitness_nq(each_solution) for each_solution in pop]
    
    fitness_best, fitness_mean = evalution(pop)
    print(f"\t{fitness_best} \t | \t{fitness_mean}")
    list_best_fitness.append(fitness_best)
    list_mean_fitness.append(fitness_mean)
    i_geracoes += 1
    

Melhor Resultado | Resultado Médio
	2 	 | 	8.8
	2 	 | 	8.2
	2 	 | 	8.1
	2 	 | 	8.1
	2 	 | 	7.7
	2 	 | 	7.3
	2 	 | 	7.4
	2 	 | 	7.0
	2 	 | 	6.9
	2 	 | 	6.6
	2 	 | 	6.1
